In [1]:
import pandas as pd

In [2]:
a50_option = pd.read_excel('/Users/50ETF期权合约基本资料.xlsx', parse_dates=True)

只要trade_code、type、到期日

In [3]:
a50_option = a50_option.loc[:, ['trade_code', 'type', 'expire']]

In [4]:
data2017 = pd.read_excel('/Users/50ETF期权日行情2017-2018.xlsx', parse_dates=True)

只要交易日期、行权价、收盘价(settle)，trade_code留作合并表格的键

In [5]:
data2017 = data2017.loc[:, ['date', 'trade_code', 'strike', 'settle']]

按trade_code合并

In [6]:
datas = pd.merge(data2017, a50_option, how='left')

In [7]:
datas.drop(columns='trade_code', inplace=True)

In [8]:
data2018 = pd.read_excel('/Users/O510050lx日数据201809至202012/201809至202012/O510050lx20180924至20201231.xlsx' , parse_dates=True)

In [9]:
data2018 = data2018.loc[:, ['报价时间','到期日','执行价','期权类型','收盘价','距离到期日天数']]

In [10]:
data2018.rename(columns = {'报价时间': 'date' , '距离到期日天数': 'T_days' , '到期日':'expire' , '期权类型':'type' , '收盘价':'settle' , '执行价':'strike'},inplace = True)

挑出call-option

In [11]:
calls = datas[datas['type'] == 'call']
call_data2018 = data2018[data2018['type'] == 'C']

挑出put-option

In [12]:
puts = datas[datas['type'] == 'put']
put_data2018 = data2018[data2018['type'] == 'P']

In [13]:
calls.rename(columns={'settle': 'call'}, inplace=True)
call_data2018.rename(columns={'settle': 'call'}, inplace=True)

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [14]:
puts.rename(columns={'settle': 'put'}, inplace=True)
put_data2018.rename(columns={'settle': 'put'}, inplace=True)

按日期、到期日、行权价为键合并

In [15]:
options = pd.merge(calls, puts.loc[:, ['date', 'expire', 'strike', 'put']], 
                   on=['date', 'expire', 'strike'], how='left').drop_duplicates()
option_data2018 = pd.merge(call_data2018, put_data2018.loc[:, ['date', 'expire', 'strike', 'put','T_days']], 
                   on=['date', 'expire', 'strike','T_days'], how='left').drop_duplicates()

In [16]:
options = options.loc[:, ['date', 'expire', 'strike', 'call', 'put']]
option_data2018 = option_data2018.loc[:, ['date', 'expire', 'strike', 'call', 'put','T_days']]

计算同一到期日、行权价的期权剩余的天数

In [17]:
options['T_days'] = (options['expire'] - options['date']).apply(lambda x: x.days)

合并options和options_data2018两个表

In [18]:
option = pd.concat([options , option_data2018])

定位第s+1小的值

In [19]:
def small_option(x , s):
    ts = list(set(x))
    ts.sort()
    return ts[s]

找到每个交易日 距离到期日最近的第1、2、3个期权的到期天数

In [20]:
last1 = option.groupby('date')['T_days'].apply(small_option, s=0)

In [21]:
last2 = option.groupby('date')['T_days'].apply(small_option, s=1)

In [22]:
last3 = option.groupby('date')['T_days'].apply(small_option, s=2)

In [23]:
last = pd.concat([last1, last2, last3], axis=1)

In [24]:
last.columns = ['last1', 'last2', 'last3']

In [25]:
for i in range(len(last)):
    if last['last1'][i] <= 7: 
        last.loc[last.index[i], 'last1'] = last.loc[last.index[i],'last2']
        last.loc[last.index[i], 'last2'] = last.loc[last.index[i],'last3']

In [26]:
options = pd.merge(option, last, on='date', how='left')

In [27]:
options['date'] = options['date'].dt.date
options['expire'] = options['expire'].dt.date

In [28]:
options.to_excel('/Users/50ETF期权已处理数据——iVIX用.xlsx')